In [2]:
import pandas as pd
from datetime import time, timedelta

# Read your data
df = pd.read_csv("17.01 UPMC Demo\soln.csv", index_col=0)

In [3]:
doctor_dfs = {}

# Function to convert minutes from start to actual time
def minutes_to_time(minutes):
    start_hour = 8  # 8 AM
    total_minutes = start_hour * 60 + minutes
    hours = (total_minutes // 60) % 24
    mins = total_minutes % 60
    return time(hours, mins)

# Split into 8 dataframes (one per doctor/machine)
for doctor_num in range(8):
    doctor_df = df[df['machine'] == doctor_num].copy()
    
    # Select and rename columns
    doctor_df = doctor_df[['job_id', 'start_time', 'end_time', 'processing_time', 'arrival_time']]
    doctor_df = doctor_df.rename(columns={'job_id': 'patient_id'})
    
    # Convert times to actual clock times
    doctor_df['start_time'] = doctor_df['start_time'].apply(minutes_to_time)
    doctor_df['end_time'] = doctor_df['end_time'].apply(minutes_to_time)
    doctor_df['arrival_time'] = doctor_df['arrival_time'].apply(minutes_to_time)
    
    # Sort by start time
    doctor_df = doctor_df.sort_values('start_time').reset_index(drop=True)
    
    # Store in dictionary with doctor number as key
    doctor_dfs[doctor_num] = doctor_df


In [6]:
# Access individual doctor schedules:
# doctor_dfs[0] for Doctor 0
# doctor_dfs[1] for Doctor 1, etc.
doctor_dfs[3]

,patient_id,start_time,end_time,processing_time,arrival_time
0,15,08:01:00,08:31:00,30,08:01:00
1,1,08:31:00,09:01:00,30,08:01:00
2,23,09:01:00,09:16:00,15,08:45:00
3,44,09:30:00,10:00:00,30,09:30:00
4,79,10:15:00,10:30:00,15,10:15:00
5,74,10:30:00,10:45:00,15,10:15:00
6,78,10:45:00,10:50:00,5,10:15:00
7,92,10:50:00,11:20:00,30,10:50:00
8,86,11:20:00,11:25:00,5,10:50:00
9,99,11:25:00,11:55:00,30,10:50:00
